In [2]:
import json

file = open("./Sarcasm_Headlines_Dataset.json")
json_data = [json.loads(line) for line in file]
json_data[0:5]

[{'article_link': 'https://www.huffingtonpost.com/entry/versace-black-code_us_5861fbefe4b0de3a08f600d5',
  'headline': "former versace store clerk sues over secret 'black code' for minority shoppers",
  'is_sarcastic': 0},
 {'article_link': 'https://www.huffingtonpost.com/entry/roseanne-revival-review_us_5ab3a497e4b054d118e04365',
  'headline': "the 'roseanne' revival catches up to our thorny political mood, for better and worse",
  'is_sarcastic': 0},
 {'article_link': 'https://local.theonion.com/mom-starting-to-fear-son-s-web-series-closest-thing-she-1819576697',
  'headline': "mom starting to fear son's web series closest thing she will have to grandchild",
  'is_sarcastic': 1},
 {'article_link': 'https://politics.theonion.com/boehner-just-wants-wife-to-listen-not-come-up-with-alt-1819574302',
  'headline': 'boehner just wants wife to listen, not come up with alternative debt-reduction ideas',
  'is_sarcastic': 1},
 {'article_link': 'https://www.huffingtonpost.com/entry/jk-rowling-w

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
import string
from bs4 import BeautifulSoup

table = str.maketrans("", "", string.punctuation)
stopwords = set(nltk.corpus.stopwords.words('english'))

In [6]:
texts = []
labels = []
urls = []

for item in json_data:
    sentence = item['headline'].lower()
    sentence = sentence.replace(",", " , ")
    sentence = sentence.replace(".", " . ")
    sentence = sentence.replace("!", " ! ")
    sentence = sentence.replace("?", " ? ")
    sentence = sentence.replace("/", " / ")
    sentence = sentence.replace("-", " - ")
    soup = BeautifulSoup(sentence)
    sentence = soup.get_text()
    words = sentence.split()
    filtered_sentence = ""
    for word in words:
      word = word.translate(table)
      if word not in stopwords:
        filtered_sentence += word + " "
    texts.append(filtered_sentence)
    labels.append(item["is_sarcastic"])
    urls.append(item["article_link"])

<ipython-input-6-b06ba4ddd63c>:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(sentence)


In [44]:
training_size = 24000

training_sentences = texts[0:training_size]
training_labels = labels[0:training_size]

testing_sentences = texts[training_size:]
testing_labels = labels[training_size:]

[0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 

In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

padding_type = "post"
trunc_type = "post"
max_length = 10

tokenizer = Tokenizer(num_words=20000, oov_token="<OOV>")
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, padding=padding_type, truncating=trunc_type, maxlen=max_length)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, padding=padding_type, truncating=trunc_type, maxlen=max_length)

In [40]:
import numpy as np

training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [33]:
embedding_dim = 12
vocab_size = 20000

model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim, return_sequences=True, dropout=0.2)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim, dropout=0.2)),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [34]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 12)          240000    
                                                                 
 bidirectional_1 (Bidirecti  (None, None, 24)          2400      
 onal)                                                           
                                                                 
 bidirectional_2 (Bidirecti  (None, 24)                3552      
 onal)                                                           
                                                                 
 dense_2 (Dense)             (None, 24)                600       
                                                                 
 dense_3 (Dense)             (None, 1)                 25        
                                                                 
Total params: 246577 (963.19 KB)
Trainable params: 246

In [35]:
adam = tf.keras.optimizers.Adam(learning_rate = 0.000008,
                                beta_1=0.9, beta_2=0.999, amsgrad=False)

model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [36]:
model.fit(training_padded, training_labels, epochs=30)

Epoch 1/30
750/750 [==============================] - 20s 17ms/step - loss: 0.6921 - accuracy: 0.5541
Epoch 2/30
750/750 [==============================] - 8s 10ms/step - loss: 0.6902 - accuracy: 0.5595
Epoch 3/30
750/750 [==============================] - 8s 10ms/step - loss: 0.6882 - accuracy: 0.5595
Epoch 4/30
750/750 [==============================] - 7s 9ms/step - loss: 0.6862 - accuracy: 0.5595
Epoch 5/30
750/750 [==============================] - 8s 10ms/step - loss: 0.6842 - accuracy: 0.5595
Epoch 6/30
750/750 [==============================] - 6s 9ms/step - loss: 0.6825 - accuracy: 0.5595
Epoch 7/30
750/750 [==============================] - 7s 10ms/step - loss: 0.6807 - accuracy: 0.5595
Epoch 8/30
750/750 [==============================] - 6s 9ms/step - loss: 0.6785 - accuracy: 0.5595
Epoch 9/30
750/750 [==============================] - 7s 10ms/step - loss: 0.6758 - accuracy: 0.5595
Epoch 10/30
750/750 [==============================] - 7s 9ms/step - loss: 0.6725 - accuracy:

In [39]:
print(testing_labels.shape)

(2709, 10)


In [46]:
testing_labels = np.array(testing_labels)
model.evaluate(testing_padded, testing_labels)

85/85 [==============================] - 2s 4ms/step - loss: 0.4839 - accuracy: 0.7715


[0.48385775089263916, 0.7715023756027222]